In [37]:
import pandas as pd
import numpy as np

# display settings
pd.options.display.max_columns = None

In [38]:
def load() -> pd.DataFrame:
    return pd.read_excel("raw_data.xlsx")

df = load()

In [43]:
df.head()

,Date Announced,Date Effective,Date Withdrawn,Target Company Date of Fin.,Target Name,Target Primary SIC Code,Target Industry Sector,Acquiror Name,Acquiror Industry Sector,Acquiror Primary SIC Code,Status,% sought,Value of Transaction ($mil),Ranking Value inc. Net Debt of Target ($Mil),Enterprise Value ($mil),Equity Value ($mil),Price Per Share,Target Net Sales LTM ($mil),EBIT Last Twelve Months ($ Mil),Pre-tax Income Last Twelve Months ($ Mil),Net Income Last Twelve Months ($ Mil),Target Net Assets ($mil),Target Total Assets ($mil),Target EBITDA LTM ($mil),Target Book Value Per Share LTM (US$),Target Common Equity ($mil),Target Earnings Per Share LTM (US$),Ratio of Offer Price to EPS,"Target and Acquiror Financial Advisor Fees, Total ($mil)",Target Share Price 1 Day Prior to Announcement ($),Ratio of Deal Value to Sales,Ratio of Deal Value to EBIT,Ratio of Deal Value to EBITDA,Ratio of Deal Value to Net Income,Ratio of Enterprise Value to Sales,Ratio of Enterprise Value to EBIT,Ratio of Enterprise Value to EBITDA,Ratio of Enterprise Value to Net Income,Equity Value/ Net Sales,Ratio of Equity Value to EBIT,Ratio of Equity Value at Announcement to EBITDA,Ratio of Equity Value to Net Income,Acquiror Cvt Debt LTM ($ mil),Acquiror Curr Assets LTM ($ mil),Acquiror Curr Liabilities LTM ($ mil),Acquiror Common Equity LTM ($ mil),Acquiror Earnings Per Share LTM ($),Acquiror EBIT LTM ($ mil),Acquiror EBITDA LTM ($ mil),Acquiror Net Assets ($mil),Acquiror Net Debt LTM ($ mil),Acquiror Net Income Last Twelve Months ($Mil),Acquiror Net Sales ($mil),Target Closing Price 1 Day After Ann Date ($),Target Ticker Symbol,Target Share Price 1 Day Prior to Announcement ($).1,Low Target Stock Price One Year Prior,S&P 500 Closing Price 1 Day After Announcement,Price Per Share.1,S&P 500 Closing Price 1 Day Before Announcement,Target R&D One Year Prior ($ mil),Repurchase,Target Debt Ratio,Target Debt Ratio Two Years Prior,Price / Net Assets Per Share,Share- holders Equity/ Total Assets,P-E Ratio 4 weeks prior to ann. date,Rankval To Sales,Rankval To Net Income,Rankval To Net Assets,Ratio of Rank Value to Forecasted Sales Current Fiscal Year,Ratio of Rank Value to Forecasted Sales Next Twelve Months,Ratio of Rank Value to Forecasted EBIT Current Fiscal Year,Ratio of Rank Value to Forecasted EBIT Next Twelve Months,Ratio of Rank Value to Forecasted EBITDA Current Fiscal Year,Ratio of Rank Value to Forecasted EBITDA Next Twelve Months,Rankval To EBIT,Rankval To EBITDA,Target Return On Assets LTM,Return On Total Equity,Ratio of Offer Price to Book Value,Target Profit Margin,Ratio of Price Book Premium to Book Value,Ratio of Offer Price to EPS.1,Target Return On Equity LTM,Target Return On Equity One Year Prior,Target Debt/ Capital.,Target Long- Term Debt/ Share- holders Equity,Long Term Debt/ Total Assets,Firm Value/ EBIT,Firm Value/ EBITDA,Firm Value/ Sales,Implied Value/ Target Common Equity,Implied Value/ Target Net Income,Implied Value/ Target Pre-tax Income,Firm Value/ Assets- Cash,Equity Value/ Pre-Tax Net Income Common Stock,Ratio of Equity Value to Forecasted Sales Next Twelve Months,Ratio of Equity Value to Forecasted Sales Current Fiscal Year,Equity Value/ Net Sales.1,Ratio of Equity Value to Forecasted EBITDA Current Fiscal Year,Ratio of Equity Value to Forecasted EBITDA Next Twelve Months,Ratio of Equity Value to Net Income.1,Ratio of Equity Value to Forecasted EBIT Current Fiscal Year,Ratio of Equity Value to Forecasted EBIT Next Twelve Months,Equity Value/ Common Equity,Ebit / Assets 3 Year Growth Rate,EBIT / Return on Assets,Ebit / Sales 3 Year Growth Rate,EBITDA ROA,Value / Net Assets,Ratio of Equity Value to EBIT.1,Ratio of Equity Value at Announcement to EBITDA.1,Ratio of Deal Value to Forecasted Sales Next Twelve Months,Ratio of Deal Value to Forecasted Sales Year 2,Common Equity Multiple,% seeking to own after trans- action,Offer Price to Target Stock Price Premium 1 Day Prior to Announcement,Number of days from Announcement Date to Co

## ETL

### Clean

In [39]:
df.columns = df.columns.str.replace(r'\s+', " ", regex=True).str.strip()
df = df.applymap(lambda x: np.nan if x in ["np", "nm", "nan"] else x)

### Filter

In [40]:
def apply_filters(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy(deep=True)
    # filter for only completed/failed mergers
    df = df[df["Status"].isin(["Completed", "Withdrawn"])]
    
    # remove share buybacks (target and acquiror are same company)
    df = df[df["Target Name"] != df["Acquiror Name"]]
    df = df[df["Date Announced"] != df["Date Effective"]]
    df = df[df["Date Announced"] != df["Date Withdrawn"]]
    df = df[df["Repurchase"] == "No"]
    return df

df_clean = apply_filters(df)

In [41]:
'''
def check_enough_entries(df: pd.DataFrame, col: str) -> bool:
    # return false if col has >30% nans
    c = df[col]
    return ( c.isin([np.nan]).sum() / len(c) ) <= 0.30
new_cols = [c for c in df.columns if check_enough_entries(df, c)]
'''

'\ndef check_enough_entries(df: pd.DataFrame, col: str) -> bool:\n    # return false if col has >30% nans\n    c = df[col]\n    return ( c.isin([np.nan]).sum() / len(c) ) <= 0.30\nnew_cols = [c for c in df.columns if check_enough_entries(df, c)]\n'

In [44]:
dt_cols = [
    "Date Announced",
    "Date Effective",
    "Date Withdrawn",
]

str_cols = [
    "Deal Number",
    "Target Name",
    "Target Ticker Symbol",
    "Target CUSIP",
    "Target Industry Sector",
    "Acquiror Name",
    "Acquiror Ticker Symbol",
    "Acquiror CUSIP",
    "Acquiror Industry Sector",
]

num_cols_generic = [
    "% sought",
    "Value of Transaction ($mil)",
    "Enterprise Value ($mil)",
    "Equity Value ($mil)",
    "Ratio of Offer Price to EPS",
    "Target and Acquiror Financial Advisor Fees, Total ($mil)",
    "Target Share Price 1 Day Prior to Announcement ($)",
    "Ratio of Deal Value to Sales",
    "% seeking to own after trans- action",
    "Exchange Ratio",
]

num_cols_target = [
    "Equity Value at Announcement ($mil)",
    "Target Net Sales LTM ($mil)",
    "EBIT Last Twelve Months ($ Mil)",
    "Pre-tax Income Last Twelve Months ($ Mil)",
    "Net Income Last Twelve Months ($ Mil)",
    "Target Net Assets ($mil)",
    "Target Total Assets ($mil)",
    "Target EBITDA LTM ($mil)",
    "Net Debt ($ mil)",
    "Target Book Value Per Share LTM (US$)",
    "Target Common Equity ($mil)",
    "Target Earnings Per Share LTM (US$)",
    "Target R&D One Year Prior ($ mil)",
    "Target Debt Ratio",
    "Target Debt Ratio Two Years Prior",
    "Price / Net Assets Per Share",
    "Share- holders Equity/ Total Assets",
    "Target Return On Assets LTM",
    "Return On Total Equity",
    "Target Return On Equity LTM",
    "Target Return On Equity One Year Prior",
    "Target Debt/ Capital.",
    "Target Long- Term Debt/ Share- holders Equity",
    "Long Term Debt/ Total Assets",
    "Firm Value/ EBIT",	
    "Firm Value/ EBITDA",
    "Firm Value/ Sales",
    "Implied Value/ Target Common Equity",
    "Firm Value/ Assets- Cash",
    "Equity Value/ Common Equity",
    "Value / Net Assets",
    "Common Equity Multiple",
    "Ratio of Offer Price to EPS",
    "Ratio of Deal Value to Sales",
    "Ratio of Deal Value to EBIT",
    "Ratio of Deal Value to EBITDA",
    "Ratio of Deal Value to Net Income",
    "Ratio of Enterprise Value to Sales",
    "Ratio of Enterprise Value to EBIT",
    "Ratio of Enterprise Value to EBITDA",
    "Ratio of Enterprise Value to Net Income",
    "Equity Value/ Net Sales",
    "Ratio of Equity Value to EBIT",
    "Ratio of Equity Value at Announcement to EBITDA",
    "Ratio of Equity Value to Net Income",
]

num_cols_acq = [
    "Acquiror Cvt Debt LTM ($ mil)",
    "Acquiror Curr Assets LTM ($ mil)",
    "Acquiror Curr Liabilities LTM ($ mil)",
    "Acquiror Common Equity LTM ($ mil)",
    "Acquiror Earnings Per Share LTM ($)",
    "Acquiror EBIT LTM ($ mil)",
    "Acquiror EBITDA LTM ($ mil)",
    "Acquiror Net Assets ($mil)",
    "Acquiror Net Debt LTM ($ mil)",
    "Acquiror Net Income Last Twelve Months ($Mil)",
    "Acquiror Net Sales ($mil)",
]

closepx_cols = [
    "Target Share Price 1 Day Prior to Announcement ($)",
    "Target Closing Price 1 Day After Ann Date ($)",
    "S&P 500 Closing Price 1 Day After Announcement",
    "S&P 500 Closing Price 1 Day Before Announcement",
    "Price Per Share",
    "Acquiror Closing Price 90 Days After Ann Date ($)",
    "Target Closing Price 90 Days After Ann Date ($)",
    "Target Share Price 1 Week Prior to Announcement ($)",
    "Target Share Price 4 Weeks Prior to Announcement ($)",
    "Premium 1 day prior to announce- ment date",
    "Premium 1 week prior to announce- ment date",
    "Premium 4 weeks prior to ann. date",
] 

y = ["Status"]

In [45]:
dt_df = df_clean[dt_cols].apply(pd.to_datetime)
str_df = df_clean[str_cols]
numeric_df = df_clean[num_cols_generic + num_cols_target + num_cols_acq].astype(float)
closepx_df = df_clean[closepx_cols].astype(float)
acquiror_is_public = (~df_clean["Acquiror Ticker Symbol"].isnull()).rename("Acquiror is public")
y_true = df_clean[y] == "Completed"

In [46]:
out = pd.concat([dt_df, str_df, numeric_df, closepx_df, acquiror_is_public, y_true], axis=1)
out = out.rename(columns={"Status":"Success"})
out = out.loc[:,~out.columns.duplicated()].copy()
out.shape

(1374, 89)

In [47]:
out.head()

,Date Announced,Date Effective,Date Withdrawn,Deal Number,Target Name,Target Ticker Symbol,Target CUSIP,Target Industry Sector,Acquiror Name,Acquiror Ticker Symbol,Acquiror CUSIP,Acquiror Industry Sector,% sought,Value of Transaction ($mil),Enterprise Value ($mil),Equity Value ($mil),Ratio of Offer Price to EPS,"Target and Acquiror Financial Advisor Fees, Total ($mil)",Target Share Price 1 Day Prior to Announcement ($),Ratio of Deal Value to Sales,% seeking to own after trans- action,Exchange Ratio,Equity Value at Announcement ($mil),Target Net Sales LTM ($mil),EBIT Last Twelve Months ($ Mil),Pre-tax Income Last Twelve Months ($ Mil),Net Income Last Twelve Months ($ Mil),Target Net Assets ($mil),Target Total Assets ($mil),Target EBITDA LTM ($mil),Net Debt ($ mil),Target Book Value Per Share LTM (US$),Target Common Equity ($mil),Target Earnings Per Share LTM (US$),Target R&D One Year Prior ($ mil),Target Debt Ratio,Target Debt Ratio Two Years Prior,Price / Net Assets Per Share,Share- holders Equity/ Total Assets,Target Return On Assets LTM,Return On Total Equity,Target Return On Equity LTM,Target Return On Equity One Year Prior,Target Debt/ Capital.,Target Long- Term Debt/ Share- holders Equity,Long Term Debt/ Total Assets,Firm Value/ EBIT,Firm Value/ EBITDA,Firm Value/ Sales,Implied Value/ Target Common Equity,Firm Value/ Assets- Cash,Equity Value/ Common Equity,Value / Net Assets,Common Equity Multiple,Ratio of Deal Value to EBIT,Ratio of Deal Value to EBITDA,Ratio of Deal Value to Net Income,Ratio of Enterprise Value to Sales,Ratio of Enterprise Value to EBIT,Ratio of Enterprise Value to EBITDA,Ratio of Enterprise Value to Net Income,Equity Value/ Net Sales,Ratio of Equity Value to EBIT,Ratio of Equity Value at Announcement to EBITDA,Ratio of Equity Value to Net Income,Acquiror Cvt Debt LTM ($ mil),Acquiror Curr Assets LTM ($ mil),Acquiror Curr Liabilities LTM ($ mil),Acquiror Common Equity LTM ($ mil),Acquiror Earnings Per Share LTM ($),Acquiror EBIT LTM ($ mil),Acquiror EBITDA LTM ($ mil),Acquiror Net Assets ($mil),Acquiror Net Debt LTM ($ mil),Acquiror Net Income Last Twelve Months ($Mil),Acquiror Net Sales ($mil),Target Closing Price 1 Day After Ann Date ($),S&P 500 Closing Price 1 Day After Announcement,S&P 500 Closing Price 1 Day Before Announcement,Price Per Share,Acquiror Closing Price 90 Days After Ann Date ($),Target Closing Price 90 Days After Ann Date ($),Target Share Price 1 Week Prior to Announcement ($),Target Share Price 4 Weeks Prior to Announcement ($),Premium 1 day prior to announce- ment date,Premium 1 week prior to announce- ment date,Premium 4 weeks prior to ann. date,Acquiror is public,Success
0,2017-01-03,NaT,2017-01-23,3056656020,InVivo Therapeutics Holdings,NVIV,46186M,"Measuring, Medical, Photo Equipment; Clocks",PixarBio Corp,PXRB,0F1095,Amusement and Recreation Services,100.0,77.000,NaN,NaN,NaN,NaN,4.20,NaN,100.0,NaN,NaN,NaN,-24.063,-23.4,-23.4,28.9,34.8,-23.510,-32.127,677.6,28.949,-566.5814,12.6,0.2,0.2,2.7,83.23,-67.38,NaN,NaN,-80.96,0.03,0.03,2.45,NaN,NaN,NaN,NaN,NaN,NaN,2.7,2.660,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.45,2270.75,2238.83,NaN,NaN,4.00,4.35,4.75,NaN,NaN,NaN,True,False
1,2017-01-03,2017-04-24,NaT,3056489020,Kopin Corp,KOPN,500600,Electronic and Electrical Equipment,Goertek Inc,002241,37883H,Electronic and Electrical Equipment,11.4,24.700,139.803,217.001,NaN,NaN,2.84,9.584,11.4,NaN,217.001,22.643,-28.173,-19.9,-23.4,74.2,87.8,-27.179,-77.198,1.1,74.044,-0.3659,16.0,0.2,0.1,2.9,84.30,-26.68,NaN,NaN,-31.65,NaN,NaN,NaN,NaN,NaN,6.18,NaN,13.16,2.931,2.9,2.931,NaN,NaN,NaN,6.174,NaN,NaN,NaN,9.584,NaN,NaN,NaN,NaN,1702.13,1327.40,1503.06,0.0779,283.25,414.85,1503.5,612.63,237.9,2778.0,3.50,2270.75,2238.83,NaN,4.90,3.98,2.96,2.73,NaN,NaN,NaN,True,True
2,2017-01-03,NaT,2017-06-16,3060281020,Tangoe Inc,TNGO,87582Y,Prepackaged Software,Tangoe Inc SPV,NaN,2F1214,"Investment & Commodity Firms,Dealers,Exchanges",75.0,207.204,244.269,275.788,NaN,

## To pickle


In [48]:
out.to_pickle("data.pickle")